TODO:

In monotonic stack:

Find where we should pop. The example values shows if it is increasing or decreasing stack.

In increasing:

We cannot extend stack in minimum values. when current value is less, we should pop all stack till we can add the new value

The calculations happend while poping

1. Balanced Brackets
Pattern: Use a stack to check if each opening bracket has a matching closing bracket in the correct order.

In [ ]:
def isValid(s):
    stack = []
    bracket_map = {')': '(', '}': '{', ']': '['}
    for char in s:
        if char in bracket_map:
            if stack and stack[-1] == bracket_map[char]:
                stack.pop()
            else:
                return False
        else:
            stack.append(char)
    return not stack


In [ ]:
s = "({[]})"
'''
Stack: [
Push '(' -> [(]
Push '{' -> [({]
Push '[' -> [({[]
Pop ']'  -> [({]
Pop '}'  -> [(]
Pop ')'  -> []
]
'''
# Result: Stack is empty, so the brackets are balanced.

2. Next Greater Element

Pattern: Use a stack to keep track of elements in decreasing order. When finding a larger element, pop from the stack to set the "next greater" for each element.

Circular Fashion (wrap around): 

one additional round is sufficient:

First Round: In the first len(nums) iterations, we process each element normally, pushing it to the stack and trying to find its next greater element using elements to its right.

Second Round: In the next len(nums) iterations, we wrap around to the beginning of the array without adding elements to the stack again (to avoid duplicates). This allows elements near the end of the array to "see" elements from the beginning, simulating the circular behavior.

Since every element has been processed in the first round, one more pass through the array is enough to make sure each element can “see” any larger element that appears earlier in the array.

In [ ]:
def nextGreaterElements(nums):
    result = [-1] * len(nums)
    stack = []
    for i in range(len(nums) * 2):
        num = nums[i % len(nums)]
        while stack and nums[stack[-1]] < num:
            result[stack.pop()] = num
        if i < len(nums):
            stack.append(i)
    return result

In [ ]:
nums = [2, 1, 3]
'''
Stack: []
Push 2  -> [2]
1 < 2, do nothing
3 > 2, Pop -> result[0] = 3

Result: [3, 3, -1]
'''

nums = [1,1,2,1,1]
'''
why 2nd loop: See end of array by wrap around (2nd round of loop)
It allows to record last elements and give chances to 2 last elements (num = 1) in stack, to compare with num = 2
Result: [2,2,-1,2,2]
'''

3. Largest Rectangle in Histogram

Pattern: Use a stack to store indices of bars in increasing order. For each bar, calculate the area by popping the stack until the current height is taller.

In [ ]:
def largestRectangleArea(heights):
    heights.append(0)  # Sentinel to clear stack at the end
    stack = []
    max_area = 0
    for i, h in enumerate(heights):
        while stack and heights[stack[-1]] > h:
            height = heights[stack.pop()]
            # The width of the rectangle with height heights[start_idx] extends from the next element after stack[-1] (i.e., stack[-1] + 1) up to cur_idx - 1
            width = i if not stack else i - stack[-1] - 1
            max_area = max(max_area, height * width)
        stack.append(i)
    return max_area


In [ ]:
heights = [2, 1, 5, 6, 2, 3]
'''
Stack: []
2 -> [0]
1 -> Pop 2, width=1 -> max_area=2 -> [1]

Result: Maximum area is 10
'''

'''
monotonic stack - increasing
            |
        |   |
    |   |   |
|   |   |   |   |   Pop happen in small current values
                    Stack is always increasing
                    Then, append an 0 value to end of height to pop at the end and go backward
'''

4. Remove K Digits to Get the Smallest Number

Pattern: Use a stack to build the smallest possible number by maintaining a monotonic increasing stack and removing elements that are larger than the current digit if there’s room to remove more.

Use Case: Given a number as a string, remove k digits to form the smallest possible number.

In [ ]:
def removeKdigits(num, k):
    stack = []
    for digit in num:
        while k > 0 and stack and stack[-1] > digit:
            stack.pop()
            k -= 1
        stack.append(digit)
    final_stack = stack[:-k] if k else stack
    # lstrip('0') removes zeros from the start (left side) of the string to produce a clean number representation, and or '0' ensures that we get "0" if the string was initially all zeros.
    return ''.join(final_stack).lstrip('0') or '0'


In [ ]:
num = "1432219", k = 3
Result: "1219"

5. Decode String (Nested Decoding)

Pattern: Use two stacks to handle nested patterns — one for numbers and one for strings. Push intermediate results and repeat counts to the stacks as you encounter opening brackets.

Use Case: Decode strings like "3[a2[c]]" to "accaccacc".

In [ ]:
def decodeString(s):
    stack = []
    num = 0
    current_str = ""
    for char in s:
        if char.isdigit():
            num = num * 10 + int(char)
        elif char == '[':
            stack.append((current_str, num))
            current_str, num = "", 0
        elif char == ']':
            last_str, repeat = stack.pop()
            current_str = last_str + current_str * repeat
        else:
            current_str += char
    return current_str


In [ ]:
s = "100[leetcode]"
'''
1: 0 * 10 + 1
0: 1 * 10 + 0
0: 10 * 10 + 0 = 100 
num = num * 10 + int(char)
'''

s = "3[a2[c]]"
'''
3: counter = 3, current_string = ''
[: push stack ['', 3]
a: current_string = a
2: counter = 2, 
[: push stack [('', 3), (a, 2)] current_string = '' counter = 0
c: current_string = c
]: pop (a,2): curret_string = a + 2 * c = acc
]: pop ('', 3): pre_string + pre_counter * current_string = '' + 3 * acc = accaccacc
'''

6. Simplify File Path (Unix Path Simplification)

Pattern: Use a stack to navigate paths by pushing valid directories and popping when encountering ...

Use Case: Simplify a Unix-style path like "/a/./b/../../c/" to "/c".

In [ ]:
def simplifyPath(path):
    stack = []
    for part in path.split('/'):
        if part == '..':
            if stack: stack.pop()
        elif part and part != '.':
            stack.append(part)
    return '/' + '/'.join(stack)


time complexity is:

O(n) for split() + O(n) for processing each component, which sums up to O(n).

In [ ]:
path = "/a/./b/../../c/"
'''
Result: "/c"
'''

7. Evaluate Infix Expression Using Two Stacks

Pattern: Use two stacks, one for values and one for operators. Process operators based on precedence.

Use Case: Evaluate expressions like "3 + 5 * (2 - 8)".

In [ ]:
'''
Basic Calculator 3
'''
class Solution:
    '''
    string = "3 + 5 * (2 - 8)"
    3: num -> num * 10 + 3
            add to value
    +: op -> add to op
    5: num -> ..
            add to val
    *: ops -> 
            calculate based on ops[-1] > * -> no
            just add to ops
    (: ops -> add to ops
    ...
    ): pop ops while != )
            pop values
            pop ops
            calculate_op
            add to values
        pop ) ops

    remaining 3 + 5 * -6
    -6 * 5 = -30
    3 + -30 = 27
    '''
    def calculate(self, s: str) -> int:
        def apply_op(left, right, op):
            if op == "+": return left + right
            elif op == "-": return left - right
            elif op == "*": return left * right 
            elif op == "/": return int(left / right)
        
        def priority_op(op):
            if op in ("-", "+"): return 1
            elif op in ("*", "/"): return 2
            return 0
        
        def calculate_op(value, operator):
            right, left = value.pop(), value.pop()
            op = operator.pop()
            calculation = apply_op(left, right, op)
            value.append(calculation)

        # stack for numbers
        value = []
        # stack for operators
        operator = []

        num = 0
        is_num = False

        for char in s + " ":
            if char.isdigit(): 
                num = num * 10 + int(char)
                is_num = True
            else:
                if is_num:
                    value.append(num)
                    num = 0
                    is_num = False
                if char == " ": continue
                elif char == "(":
                    operator.append(char)
                elif char == ")":
                    while operator and operator[-1] != "(":
                        calculate_op(value, operator)
                    # Remove the '(' from ops stack
                    operator.pop() 
                else:
                    # find if it is a priority 
                    while operator and priority_op(operator[-1]) >= priority_op(char):
                        calculate_op(value, operator)
                    operator.append(char)

        # For what is remained in operator
        while operator:
            calculate_op(value, operator)

        return value[0]
        

In [ ]:
expression = "3 + 5 * ( 2 - 8 )"
'''
Result: -27
'''

8. Basic Calculator with Parentheses

Pattern: Use a stack to handle nested calculations, especially when dealing with parentheses.

Use Case: Evaluate basic expressions with +, -, (, ) without operator precedence.

In [ ]:
def calculate(s):
    stack = []
    current_number = 0
    result = 0
    sign = 1
    
    for char in s:
        if char.isdigit():
            current_number = current_number * 10 + int(char)
        elif char == '+':
            result += sign * current_number
            sign = 1
            current_number = 0
        elif char == '-':
            result += sign * current_number
            sign = -1
            current_number = 0
        elif char == '(':
            stack.append(result)
            stack.append(sign)
            result = 0
            sign = 1
        elif char == ')':
            result += sign * current_number
            result *= stack.pop()
            result += stack.pop()
            current_number = 0

    return result + sign * current_number


In [ ]:
Example: s = "1 + (2 - (3 + 4))"
'''
Result: -4
'''